# 1. Working with Multimodal Inputs

In this notebook, we'll explore how to work with multimodal inputs (text, images, and videos) using Amazon Bedrock's Nova model. We'll build on the prompt engineering concepts from the previous notebook.

## Setting up Bedrock client for multimodal inputs

Let's set up the Bedrock client using our utility functions that support multimodal inputs:

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append('./')

from src.utils import (
    create_bedrock_client,
    invoke_with_image,
    invoke_with_video,
    extract_json_from_text,
    invoke_with_prefill,
    NOVA_LITE
)

# Create a Bedrock client
bedrock_client = create_bedrock_client()

# Default settings
TEMPERATURE = 0.0  # Lower temperature = more deterministic outputs

# Set up assets directory - updated to use root assets folder
current_dir = Path.cwd()
assets_dir = current_dir.joinpath('assets')
print('The assets are located in:', assets_dir)

# Verify the assets directory exists
if not assets_dir.exists():
    print("WARNING: Assets directory not found. Please update the path.")

## 1. Basic Image Understanding

Let's start with a simple prompt to describe an image:

In [ ]:
image_path = f'{assets_dir}/image.png'

basic_prompt = "Describe what's happening in the image above."

response = invoke_with_image(bedrock_client, image_path, basic_prompt, model_id=NOVA_LITE, temperature=TEMPERATURE)
print(response)

## 2. Structured Image Analysis

Let's add structure to our image analysis prompt:

In [ ]:
structured_image_prompt = """
## Instructions
You are an image analysis tool. 
Given the image above, provide a clear and comprehensive description following the rules below.

## Rules
- Describe the scene in detail, including people, objects, and actions
- Comment on the setting and environment
- Note any interesting or unusual elements
- Describe the image in an engaging way, as if you are a sports commentator
"""

response = invoke_with_image(bedrock_client, image_path, structured_image_prompt, model_id=NOVA_LITE, temperature=TEMPERATURE)
print(response)

## 3. Video Understanding

Now let's analyze a video:

In [ ]:
video_path = f'{assets_dir}/video.mp4'

video_prompt = """
## Instructions
You are a video analysis tool. 
Given the video above, provide a clear and comprehensive description of the video following the rules below.

## Rules
- Describe the key scenes in the video in chronological order
- Note any changes or movements that occur throughout the video
- Comment on people, objects, actions, and setting
- Describe the video in an engaging way
"""

response = invoke_with_video(bedrock_client, video_path, video_prompt, model_id=NOVA_LITE, temperature=TEMPERATURE)
print(response)

## 4. Structured JSON Output from Images with Prefilling

Let's extract structured information from an image and return it as JSON, using prefilling to guarantee proper formatting:

In [ ]:
# Define our structured analysis schema as a JSON object
image_schema = {
    "scene_type": "string (indoor/outdoor)",
    "time_of_day": "string (day/night/unclear)",
    "num_people": "number",
    "activities": ["string"],
    "objects": ["string"],
    "mood": "string (happy/sad/neutral/excited/etc.)",
    "description": "string (1-2 sentence summary)"
}

# Inject the schema directly into the prompt
json_image_prompt = f"""
## Instructions
You are an image analysis tool. 
Given the image above, analyze it carefully and extract the requested information in the schema provided below.

## Schema
{image_schema}

## Rules
- Analyze the image carefully and provide all requested information
- Return a valid and parseable JSON object inside ```json code blocks
- Do not include any explanation or text outside the JSON object
"""

# Using prefilling to ensure we get a properly formatted JSON response
prefill = """
```json
{
  "scene_type": """

# Invoke model with prefill to ensure proper JSON structure
result = invoke_with_prefill(bedrock_client, 
                            prompt=json_image_prompt, 
                            prefill=prefill, 
                            image_path=image_path,
                            model_id=NOVA_LITE, 
                            temperature=TEMPERATURE)

# Combine the prefill with the completion for the full response
full_result = prefill + result
print(full_result)

## 5. Extracting and Using the JSON Output

Let's extract the JSON from the prefilled response and use it programmatically:

In [ ]:
try:
    image_data = extract_json_from_text(full_result)
    
    # Access specific fields
    print(f"Scene type: {image_data['scene_type']}")
    print(f"Time of day: {image_data['time_of_day']}")
    print(f"Number of people: {image_data['num_people']}")
    
    print("\nActivities:")
    for activity in image_data["activities"]:
        print(f"- {activity}")
        
    print("\nObjects:")
    for obj in image_data["objects"]:
        print(f"- {obj}")
        
    print(f"\nMood: {image_data['mood']}")
    print(f"\nDescription: {image_data['description']}")
except ValueError as e:
    print(f"Error extracting JSON: {e}")

## 6. Structured Video Analysis with Prefilling

Now let's apply the same approach to video analysis, using prefilling to ensure we get properly formatted JSON output:

In [ ]:
# Define our video analysis schema as a structured data object
video_analysis_schema = {
    "duration_impression": "string (short/medium/long)",
    "num_people": "number",
    "num_animals": "number", 
    "scenes": ["string (descriptions of key moments)"],
    "action_summary": "string (1-2 sentence summary of activity)",
    "location": "string (where the video takes place)"
}

# Using an f-string to inject our schema directly into the prompt
prefill_video_prompt = f"""
## Instructions
You are a video analysis tool. 
Given the video above, analyze it carefully and extract the requested information according to the schema.

## Schema
{video_analysis_schema}

## Rules
- Analyze the video carefully and provide all requested information
- Return a valid and parseable JSON object inside ```json code blocks
- Use chain-of-thought reasoning by first describing what you see, then filling in the schema
"""

# Start the response with the JSON structure
prefill = """
```json
{
  "duration_impression": """

# Invoke model with prefill to ensure proper JSON structure
video_result = invoke_with_prefill(bedrock_client, 
                                  prompt=prefill_video_prompt, 
                                  prefill=prefill, 
                                  video_path=video_path,
                                  model_id=NOVA_LITE, 
                                  temperature=TEMPERATURE)

# Combine prefill with completion for the full response
full_video_result = prefill + video_result
print(full_video_result)

In [ ]:
# Extract and use the structured data from video analysis
try:
    video_data = extract_json_from_text(full_video_result)
    
    print(f"Video duration impression: {video_data['duration_impression']}")
    print(f"Number of people: {video_data['num_people']}")
    print(f"Number of animals: {video_data['num_animals']}")
    print(f"Location: {video_data['location']}")
    
    print("\nKey scenes:")
    for i, scene in enumerate(video_data["scenes"], 1):
        print(f"{i}. {scene}")
        
    print(f"\nAction summary: {video_data['action_summary']}")
except ValueError as e:
    print(f"Error extracting JSON: {e}")

## 7. Chain of Thought Reasoning with Images

Chain of thought reasoning helps the model work through problems step by step, which often improves accuracy:

In [ ]:
# Define our questions as a structured data object
questions = [
    "What is the weather condition in this image?",
    "What activities are people engaged in?",
    "What time of day does this appear to be?",
    "Are there any safety concerns visible?",
    "What type of environment is shown?"
]

# Build prompt with chain-of-thought reasoning
specific_questions_prompt = f"""
## Instructions
Look at the image above and answer the following questions:

{chr(10).join(f"{i+1}. {question}" for i, question in enumerate(questions))}

## Rules
- First, describe what you see in the image in detail
- Then, think through each question step by step
- Finally, provide numbered answers to match each question (1-2 sentences per answer)
"""

response = invoke_with_image(bedrock_client, image_path, specific_questions_prompt, model_id=NOVA_LITE, temperature=TEMPERATURE)
print(response)

## 8. Asking Specific Questions About Images

We can also use the model to answer specific questions about an image, which is useful for extracting targeted information:

In [ ]:
specific_question = "How many people are in this image, and what are they wearing?"

response = invoke_with_image(bedrock_client, image_path, specific_question, model_id=NOVA_LITE, temperature=TEMPERATURE)
print(response)

## 9. Building Real-world Applications

With the skills you've learned in this notebook, you can build various real-world multimodal applications such as:

1. **Visual content moderation**: Analyzing images for inappropriate content
2. **Visual search**: Finding products based on image inputs
3. **Content cataloging**: Automatically tagging and organizing media
4. **Accessibility tools**: Creating descriptions of images for visually impaired users
5. **Security applications**: Analyzing surveillance footage
6. **Educational tools**: Creating interactive learning experiences

## 10. Next Steps

In this notebook, we've explored multimodal inputs with Amazon Bedrock's Nova model:

1. Basic image understanding
2. Structured image analysis 
3. Video understanding
4. Structured JSON outputs with prefilling for guaranteed format
5. Extracting and using structured data from media analysis
6. Chain-of-thought reasoning with visual inputs
7. Asking specific questions about images and videos

Next, explore the various design patterns in the `patterns/` directory to learn about implementing GenAI patterns like prompt chaining, routing, and orchestration.